In [4]:
# Import de SparkContext du module pyspark
from pyspark import SparkContext

# Défintion d'un SparkContext
sc = SparkContext.getOrCreate()
# 打印 SparkContext 的一些信息
print("SparkContext version:", sc.version)
print("SparkContext master:", sc.master)

your 131072x1 screen size is bogus. expect trouble
23/12/30 16:14:57 WARN Utils: Your hostname, JayWin10Pc resolves to a loopback address: 127.0.1.1; using 172.17.52.247 instead (on interface eth0)
23/12/30 16:14:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 16:14:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/30 16:15:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [13]:
import os
pwd = os.getcwd()
print(pwd)
print("{}/data/miserables_full.txt".format(pwd))

/home/jayl/datascientestfile/spark
/home/jayl/datascientestfile/spark/data/miserables_full.txt


In [20]:
# rdd named miserables
miserables = sc.textFile("{}/data/miserables_full.txt".format(pwd))
miserables.take(20)


['The Project Gutenberg eBook of Les misérables Tome I: Fantine',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '',
 'Title: Les misérables Tome I: Fantine',
 '',
 '',
 'Author: Victor Hugo',
 '',
 'Release date: January 10, 2006 [eBook #17489]',
 '                Most recently updated: October 30, 2023',
 '',
 'Language: French',
 '']

In [30]:
miserables_clean = miserables.map(lambda x: x.lower())
miserables_clean = miserables_clean.map(lambda x : x.replace("."," ").replace(","," ").replace("-"," ").replace("'"," "))

miserables_clean.take(5)


['the project gutenberg ebook of les misérables tome i: fantine',
 '    ',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever  you may copy it  give it away or re use it under the terms']

In [37]:
miserables_flat  = miserables_clean.flatMap(lambda x:x.split(" "))

miserables_flat.take(20)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'les',
 'misérables',
 'tome',
 'i:',
 'fantine',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'in']

In [39]:
mots = miserables_flat.map(lambda x : (x,1)).reduceByKey(lambda x,y :x+y )
mots.take(10)

[('project', 83),
 ('gutenberg', 34),
 ('ebook', 12),
 ('of', 122),
 ('les', 1516),
 ('tome', 5),
 ('i:', 4),
 ('fantine', 185),
 ('this', 45),
 ('is', 22)]

In [54]:
# 1 转换成collection 对collection sort
mots_sorted = sorted(mots.collect(),key = lambda x: x[1],reverse=1)
# print(mots_sorted[:5])

# 2 对rdd使用sortby 再转换成collection（可选）
mots_sorted2 = mots.sortBy(lambda x:x[1],ascending=0)
print(mots_sorted2.take(5))

mots_sorted2 = mots_sorted2.collect()
# mots_sorted2[:5]


[('de', 4469), ('il', 3161), ('la', 3024), ('et', 2943), ('le', 2534)]


In [66]:
mots_sorted3 = sc.textFile("{}/data/miserables_full.txt".format(pwd)) \
                .map(lambda x: x.lower().replace("."," ").replace(","," ").replace("-"," ").replace("'"," "))\
                .flatMap(lambda x:x.split(" "))\
                .map(lambda x:(x,1))\
                .reduceByKey(lambda x,y: x+y)\
                .sortBy(lambda x:x[1],ascending=0)


mots_sorted3.take(5)
# 为什么结果里面会有空格？ 我可以加filter函数，但是我需要知道为什么？

[('', 22131), ('de', 4469), ('il', 3161), ('la', 3024), ('et', 2943)]

In [ ]:
sc.stop()